<a href="https://colab.research.google.com/github/YolandaMDavis/cs109a-fall2020/blob/socio-economic/HousingValuesGeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pygeohash

  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6162 sha256=18f533d8c1691d119a51a411797f78c953538be0155435955defaa20afd0c611
  Stored in directory: /root/.cache/pip/wheels/3f/5f/14/989d83a271207dda28232746d63e737a2dbd88ea7f7a9db807
Successfully built pygeohash


In [3]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pygeohash as pgh
DATA_ROOT = '/content/drive/My Drive/Final-Project-for-CS109a/'

In [4]:
SAM_addresses = pd.read_csv(DATA_ROOT + 'Live_Street_Address_Management__SAM__Addresses.csv')
assessment = pd.read_csv(DATA_ROOT + '2020-property-assessment.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,9,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,15,27,28,29,34,35,36,37,38,39,40,41,42,44,45,46,47,48,58,59,64,65,66,67,68,69,70,72,73,74) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
SAM_addresses.shape

(399248, 27)

In [6]:
assessment.shape

(175052, 75)

In [7]:
SAM_addresses[SAM_addresses['PARCEL']==2200078000]

,X,Y,SAM_ADDRESS_ID,RELATIONSHIP_TYPE,BUILDING_ID,FULL_ADDRESS,STREET_NUMBER,IS_RANGE,RANGE_FROM,RANGE_TO,UNIT,FULL_STREET_NAME,STREET_ID,STREET_PREFIX,STREET_BODY,STREET_SUFFIX_ABBR,STREET_FULL_SUFFIX,STREET_SUFFIX_DIR,STREET_NUMBER_SORT,MAILING_NEIGHBORHOOD,ZIP_CODE,X_COORD,Y_COORD,SAM_STREET_ID,WARD,PRECINCT_WARD,PARCEL
132036,-71.124341,42.360299,150718,1,143177,30 Windom St,30,0,NaN,NaN,NaN,Windom St,4535,,Windom,St,Street,,30.0,Allston,2134.0,757696.825869,2.956542e+06,4535,22,2201,2.200078e+09


In [8]:
assessment[assessment['PID']==2200078000]

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,ST_NAME_SUF,UNIT_NUM,ZIPCODE,PTYPE,LU,OWN_OCC,OWNER,MAIL_ADDRESSEE,MAIL_ADDRESS,MAIL CS,MAIL_ZIPCODE,AV_LAND,AV_BLDG,AV_TOTAL,GROSS_TAX,LAND_SF,YR_BUILT,YR_REMOD,GROSS_AREA,LIVING_AREA,NUM_FLOORS,STRUCTURE_CLASS,R_BLDG_STYL,R_ROOF_TYP,R_EXT_FIN,R_TOTAL_RMS,R_BDRMS,R_FULL_BTH,R_HALF_BTH,R_BTH_STYLE,R_BTH_STYLE2,R_BTH_STYLE3,R_KITCH,R_KITCH_STYLE,R_KITCH_STYLE2,R_KITCH_STYLE3,R_HEAT_TYP,R_AC,R_FPLACE,R_EXT_CND,R_OVRALL_CND,R_INT_CND,R_INT_FIN,R_VIEW,S_NUM_BLDG,S_BLDG_STYL,S_UNIT_RES,S_UNIT_COM,S_UNIT_RC,S_EXT_FIN,S_EXT_CND,U_BASE_FLOOR,U_NUM_PARK,U_CORNER,U_ORIENT,U_TOT_RMS,U_BDRMS,U_FULL_BTH,U_HALF_BTH,U_BTH_STYLE,U_BTH_STYLE2,U_BTH_STYLE3,U_KITCH_TYPE,U_KITCH_STYLE,U_HEAT_TYP,U_AC,U_FPLACE,U_INT_FIN,U_INT_CND,U_VIEW
163062,2200078000,NaN,2200078000,30,WINDOM,ST,NaN,2134.0,101,R1,Y,ANDERSON PRISCILLA R,C/O PRISCILLA ANDERSON,30 WINDOM,ALLSTON MA,2134,189400,350700,540100,5703.45,3400,1900.0,2013.0,1940.0,1128.0,2.0,NaN,CL - Colonial,L - Gambrel,M - Vinyl,6.0,3.0,1.0,1.0,M - Modern,M - Modern,NaN,1F - 1 Full Eat In K,M - Modern,NaN,NaN,F - Forced Hot Air,N - None,0.0,A - Average,A - Average,G - Good,N - Normal,A - Average,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
assessment_SAM = pd.merge(assessment, SAM_addresses, left_on='PID',right_on="PARCEL")

In [10]:
assessment_SAM.shape

(406162, 102)

In [11]:
assessment_SAM_reduced = assessment_SAM[['PID', 'AV_LAND', 'AV_BLDG', 'AV_TOTAL', 'X', 'Y']]

In [12]:
assessment_SAM_reduced.shape

(406162, 6)

In [13]:
assessment_SAM_reduced["GEOHASH"] = [pgh.encode(row["Y"],row["X"]) for idx, row in assessment_SAM_reduced.iterrows()]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
assessment_SAM_reduced.head()

,PID,AV_LAND,AV_BLDG,AV_TOTAL,X,Y,GEOHASH
0,100272000,349800,355900,705700,-71.027291,42.380929,drt3pepwb1tx
1,100272000,349800,355900,705700,-71.027291,42.380929,drt3pepwb1tx
2,100272000,349800,355900,705700,-71.027291,42.380929,drt3pepwb1tx
3,100272000,349800,355900,705700,-71.027291,42.380929,drt3pepwb1tx
4,100274000,0,745300,745300,-71.027641,42.380959,drt3pepqbp29


In [15]:
pgh.geohash_haversine_distance(assessment_SAM_reduced.iloc[0,6], assessment_SAM_reduced.iloc[4, 6])

28.940991081356703

In [16]:
from geopy import distance

In [17]:
place1 = (assessment_SAM_reduced.iloc[0,5], assessment_SAM_reduced.iloc[0,4])
place2 = (assessment_SAM_reduced.iloc[4,5], assessment_SAM_reduced.iloc[4,4])
print(distance.distance(place1, place2).m)

29.016173010195022


In [18]:
origin = assessment_SAM_reduced[assessment_SAM_reduced['PID']==2200078000]['GEOHASH'].values[0]
origin

'drt2vx2ydspb'

In [19]:
boston_crime = pd.read_csv(DATA_ROOT + 'data_prelim/boston_crime_socio_eco.csv')
boston_crime.shape

(523518, 23)

In [20]:
# boston_crime_2020.drop(columns=['Unnamed: 0'], inplace=True)
boston_crime.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,Location,WHITE,BLACK,HISPANIC,ASIAN,OTHER,16-19-EMPLOYMENT,20-34-EMPLOYMENT,MEDIAN_INCOME
0,S97333701,3301,VERBAL DISPUTE,C6,915,0,2020-07-18 14:34:00,2020,7,Saturday,14,MARY BOYLE WAY,42.330813,-71.051368,"(42.33081300, -71.05136800)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
1,I20210846,3201,PROPERTY - LOST/ MISSING,C6,206,0,2020-02-09 00:30:00,2020,2,Sunday,0,SEAPORT BLVD,42.352756,-71.047481,"(42.35275600, -71.04748100)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
2,I20207883,3115,INVESTIGATE PERSON,C6,914,0,2020-04-18 17:11:00,2020,4,Saturday,17,OLD COLONY AVE,42.325573,-71.052481,"(42.32557300, -71.05248100)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
3,I192078622,619,LARCENY ALL OTHERS,C6,201,NaN,2019-09-29 03:04:00,2019,9,Sunday,3,W BROADWAY,42.340070,-71.052794,"(42.34006986, -71.05279420)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555
4,I192078606,3201,PROPERTY - LOST,C6,200,NaN,2019-09-29 02:21:00,2019,9,Sunday,2,W BROADWAY,42.341288,-71.054679,"(42.34128751, -71.05467933)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555


In [22]:
boston_crime["GEOHASH"] = [pgh.encode(row["Lat"],row["Long"]) for idx, row in boston_crime.iterrows()]

In [23]:
# boston_crime_2020_reduced = boston_crime_2020.dropna(axis=0, subset=['LATITUDE', 'LONGITUDE'])
# boston_crime_2020_reduced.shape
boston_crime.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,Location,WHITE,BLACK,HISPANIC,ASIAN,OTHER,16-19-EMPLOYMENT,20-34-EMPLOYMENT,MEDIAN_INCOME,GEOHASH
0,S97333701,3301,VERBAL DISPUTE,C6,915,0,2020-07-18 14:34:00,2020,7,Saturday,14,MARY BOYLE WAY,42.330813,-71.051368,"(42.33081300, -71.05136800)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555,drt2z4jfz5kp
1,I20210846,3201,PROPERTY - LOST/ MISSING,C6,206,0,2020-02-09 00:30:00,2020,2,Sunday,0,SEAPORT BLVD,42.352756,-71.047481,"(42.35275600, -71.04748100)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555,drt2zq0f9tyq
2,I20207883,3115,INVESTIGATE PERSON,C6,914,0,2020-04-18 17:11:00,2020,4,Saturday,17,OLD COLONY AVE,42.325573,-71.052481,"(42.32557300, -71.05248100)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555,drt2z1jhm4wj
3,I192078622,619,LARCENY ALL OTHERS,C6,201,NaN,2019-09-29 03:04:00,2019,9,Sunday,3,W BROADWAY,42.340070,-71.052794,"(42.34006986, -71.05279420)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555,drt2z5ubvchd
4,I192078606,3201,PROPERTY - LOST,C6,200,NaN,2019-09-29 02:21:00,2019,9,Sunday,2,W BROADWAY,42.341288,-71.054679,"(42.34128751, -71.05467933)",0.79193,0.045022,0.069712,0.074954,0.018382,0.38985,0.888016,121877.555,drt2z5gxcunf


In [30]:
# define function to apply across the GEOHASH of 
from collections import defaultdict

# setting up home value dict (to 600 meter precision) with geohash. 100 meters was giving too many 0 values (almost 1% of rows)
home_value_dict = defaultdict(list)
for idx, row in assessment_SAM_reduced.iterrows():
    home_value_dict[row['GEOHASH'][:6]].append(row['AV_TOTAL'])


In [31]:
# function for getting values from the dictionary from geohash
from statistics import mean

def get_prop_values(geohash, home_values):
  try:
    mean_property_value = mean(home_values[geohash[:6]])
  except:
    mean_property_value = 0

  return mean_property_value

In [32]:
# prop_values_test = get_prop_values(origin, home_value_dict)
# this should create list of the property values near 
LOCAL_PROP_VALUES = [get_prop_values(row["GEOHASH"], home_value_dict) for idx, row in boston_crime.iterrows()]

In [33]:
boston_crime['LOCAL_PROP_VALUES'] = LOCAL_PROP_VALUES

In [34]:
boston_crime.tail(10)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,Location,WHITE,BLACK,HISPANIC,ASIAN,OTHER,16-19-EMPLOYMENT,20-34-EMPLOYMENT,MEDIAN_INCOME,GEOHASH,LOCAL_PROP_VALUES
523508,192078695,3114,INVESTIGATE PROPERTY,D4,624,0,2019-09-29 11:29:00,2019,9,Sunday,11,BOYLSTON ST,42.345447,-71.094598,"(42.34544700, -71.09459800)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,drt2yhy2pg3q,1.635523e+07
523509,192078679,3115,INVESTIGATE PERSON,D4,,0,2019-09-29 10:26:00,2019,9,Sunday,10,MONSIGNOR REYNOLDS WAY,42.340486,-71.069716,"(42.34048600, -71.06971600)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,drt2ygbe5wdv,2.688223e+07
523510,192071899,3125,WARRANT ARREST - BOSTON WARRANT (MUST BE SUPPL...,D4,,0,2019-10-12 15:00:00,2019,10,Saturday,15,NaN,42.335002,-71.075060,"(42.33500200, -71.07506000)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,drt2yduer065,1.125403e+07
523511,192068048,801,ASSAULT - SIMPLE,D4,134,0,2019-10-04 11:30:00,2019,10,Friday,11,BOYLSTON ST,42.350378,-71.076261,"(42.35037800, -71.07626100)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,drt2yteug4dt,3.907033e+07
523512,192059632,3125,WARRANT ARREST - BOSTON WARRANT (MUST BE SUPPL...,D4,,0,2020-02-23 16:54:00,2020,2,Sunday,16,NaN,42.333496,-71.073388,"(42.33349600, -71.07338800)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,drt2ydtfqp8u,1.125403e+07
523513,192040735,3125,WARRANT ARREST - BOSTON WARRANT (MUST BE SUPPL...,D4,150,0,2019-09-30 15:30:00,2019,9,Monday,15,APPLETON ST,42.345393,-71.073802,"(42.34539300, -71.07380200)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,drt2ystxu3kh,5.518427e+06
523514,192022651,3126,WARRANT ARREST - OUTSIDE OF BOSTON WARRANT,D4,,0,2019-10-15 09:11:00,2019,10,Tuesday,9,NaN,42.333496,-71.073388,"(42.33349600, -71.07338800)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,drt2ydtfqp8u,1.125403e+07
523515,190536172,801,ASSAULT - SIMPLE,D4,152,0,2019-10-09 10:00:00,2019,10,Wednesday,10,WARREN AVE,42.344539,-71.073587,"(42.34453900, -71.07358700)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,drt2ystfc6hq,5.518427e+06
523516,190258751,3125,WARRANT ARREST - BOSTON WARRANT (MUST BE SUPPL...,D4,,0,2019-10-23 17:41:00,2019,10,Wednesday,17,NORTHAMPTON ST,42.333951,-71.075389,"(42.33395100, -71.07538900)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,drt2ydsmpkkw,1.125403e+07
523517,120200242,3115,INVESTIGATE PERSON,D4,167,0,2020-01-05 00:00:00,2020,1,Sunday,0,HARRISON AVE,42.339541,-71.069408,"(42.33954100, -71.06940800)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,drt2yg8vdy2r,2.688223e+07


In [45]:
boston_crime_reduced = boston_crime[boston_crime['Lat']!=0]
boston_crime_reduced.dropna(subset=['Lat', 'Long'], inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:
boston_crime_reduced[boston_crime_reduced['LOCAL_PROP_VALUES']==0]

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,Location,WHITE,BLACK,HISPANIC,ASIAN,OTHER,16-19-EMPLOYMENT,20-34-EMPLOYMENT,MEDIAN_INCOME,GEOHASH,LOCAL_PROP_VALUES
245,I192075099,3201,PROPERTY - LOST,C6,207,NaN,2019-09-18 13:46:00,2019,9,Wednesday,13,PIER 4 BLVD,-1.000000,-1.000000,"(-1.00000000, -1.00000000)",0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550,7zz631zyd63z,0.0
358,I192073218,3115,INVESTIGATE PERSON,C6,207,NaN,2019-09-12 13:03:00,2019,9,Thursday,13,PIER 4 BLVD,-1.000000,-1.000000,"(-1.00000000, -1.00000000)",0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550,7zz631zyd63z,0.0
1141,I192061036,3803,M/V ACCIDENT - PERSONAL INJURY,C6,,NaN,2019-08-05 18:48:00,2019,8,Monday,18,NaN,-1.000000,-1.000000,"(-1.00000000, -1.00000000)",0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550,7zz631zyd63z,0.0
1511,I192055819,3831,M/V - LEAVING SCENE - PROPERTY DAMAGE,C6,,NaN,2019-07-19 16:50:00,2019,7,Friday,16,SOUTHBAY,-1.000000,-1.000000,"(-1.00000000, -1.00000000)",0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550,7zz631zyd63z,0.0
2199,I192045328,3115,INVESTIGATE PERSON,C6,207,NaN,2019-06-15 08:00:00,2019,6,Saturday,8,PIER 4 BLVD,-1.000000,-1.000000,"(-1.00000000, -1.00000000)",0.791930,0.045022,0.069712,0.074954,0.018382,0.389850,0.888016,121877.5550,7zz631zyd63z,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453440,202053157,3116,HARBOR INCIDENT / VIOLATION,A7,,0.0,2020-07-27 13:05:00,2020,7,Monday,13,DEER ISLAND RD,42.350244,-70.953726,"(42.35024400, -70.95372600)",0.325667,0.025571,0.574151,0.038067,0.036545,0.308054,0.855628,52935.3600,drt8cmth43ej,0.0
454367,202032124,3116,HARBOR INCIDENT / VIOLATION,A7,,0,2020-05-08 09:45:00,2020,5,Friday,9,DEER ISLAND RD,42.350244,-70.953726,"(42.35024400, -70.95372600)",0.325667,0.025571,0.574151,0.038067,0.036545,0.308054,0.855628,52935.3600,drt8cmth43ej,0.0
458002,I192067942,727,AUTO THEFT - LEASED/RENTED VEHICLE,D4,109,NaN,2019-08-25 00:00:00,2019,8,Sunday,0,JERSEY,-1.000000,-1.000000,"(-1.00000000, -1.00000000)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,7zz631zyd63z,0.0
471410,I182061252,802,ASSAULT SIMPLE - BATTERY,D4,,NaN,2018-08-02 20:09:00,2018,8,Thursday,20,DITMUS,-1.000000,-1.000000,"(-1.00000000, -1.00000000)",0.654461,0.062627,0.107735,0.144875,0.030302,0.417109,0.725266,65903.7975,7zz631zyd63z,0.0


In [47]:
boston_crime_reduced['LOCAL_PROP_VALUES'].describe()

count    4.941320e+05
mean     1.098376e+07
std      2.136595e+07
min      0.000000e+00
25%      9.348540e+05
50%      2.949283e+06
75%      1.125403e+07
max      5.811994e+08
Name: LOCAL_PROP_VALUES, dtype: float64

In [49]:
# though I investigated the data by taking out NaN and 0, I'm writing all rows, even with NaNs, etc.
boston_crime.to_csv(DATA_ROOT + 'data_prelim/boston_crime_james.csv', index=False)